In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_model import ARMA

In [2]:
from weather_api import *

Za pomoca skryptu shelowego pobieramy dane dla konkretnego miasta i dostarczamy je w formie DataFrame. DataFrame został od razu "odchudzony" z danych nie wnoszących żadnych informacji (kolumny zawierające głównie wartości null).

In [3]:
df = get_data("zakopane")
df

,Kod stacji,Nazwa stacji,Rok,Miesiąc,Dzień,Średnie dobowe zachmurzenie ogólne [oktanty],Średnia dobowa prędkość wiatru [m/s],Średnia dobowa temperatura [°C],Średnia dobowe ciśnienie pary wodnej [hPa],Średnia dobowa wilgotność względna [%],Średnia dobowe ciśnienie na poziomie stacji [hPa],Średnie dobowe ciśnienie na pozimie morza [hPa],Status pomiaru PPPM,Suma opadu dzień [mm],Suma opadu noc [mm]
0,349190625,ZAKOPANE,2020,1,1,4.3,2.0,-1.7,4.0,73.1,929.5,0.0,8,0.0,0.0
1,349190625,ZAKOPANE,2020,1,2,4.4,1.3,-1.5,3.1,56.8,928.8,0.0,8,0.0,0.0
2,349190625,ZAKOPANE,2020,1,3,2.5,1.4,-1.1,3.8,68.8,922.4,0.0,8,0.0,0.0
3,349190625,ZAKOPANE,2020,1,4,7.6,3.6,0.8,5.4,83.6,917.5,0.0,8,1.9,6.5
4,349190625,ZAKOPANE,2020,1,5,7.9,2.8,-4.1,3.8,83.6,924.4,0.0,8,3.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,349190625,ZAKOPANE,2019,12,27,7.9,2.1,-2.0,4.8,91.3,920.5,0.0,8,4.6,6.5
543,349190625,ZAKOPANE,2019,12,28,8.0,2.6,-4.6,4.0,90.6,928.0,0.0,8,3.7,0.4
544,349190625,ZAKOPANE,2019,12,29,7.1,0.9,-7.3,3.2,88.3,933.2,0.0,8,1.1,0.0
545,349190625,ZAKOPANE,2019,12,30,6.0,1.3,-2.3,4.8,87.5,931.0,0.0,8,0.0,0.0


Wykorzystując założenie, że każda wartość większa od 0 w kolunach 'Suma opadu dzień' i 'Suma opadu noc' oznacza, że będzie padać odpowiednio w dzień i w nocy, dodajemy dwie kolumny z wartościami 0 - nie pada oraz 1 - pada.

In [4]:
value = []
data = df['Suma opadu dzień  [mm]']
data = data.to_numpy()
for row in data:
    if row == 0:
        value.append(0)
    elif row > 0:
        value.append(1)
df['deszcz w dzień']= value

value1 = []
data1 = df['Suma opadu noc   [mm]']
data1 = data1.to_numpy()
for row1 in data1:
    if row1 == 0:
        value1.append(0)
    elif row1 > 0:
        value1.append(1)
df['deszcz w nocy']= value1

df

,Kod stacji,Nazwa stacji,Rok,Miesiąc,Dzień,Średnie dobowe zachmurzenie ogólne [oktanty],Średnia dobowa prędkość wiatru [m/s],Średnia dobowa temperatura [°C],Średnia dobowe ciśnienie pary wodnej [hPa],Średnia dobowa wilgotność względna [%],Średnia dobowe ciśnienie na poziomie stacji [hPa],Średnie dobowe ciśnienie na pozimie morza [hPa],Status pomiaru PPPM,Suma opadu dzień [mm],Suma opadu noc [mm],deszcz w dzień,deszcz w nocy
0,349190625,ZAKOPANE,2020,1,1,4.3,2.0,-1.7,4.0,73.1,929.5,0.0,8,0.0,0.0,0,0
1,349190625,ZAKOPANE,2020,1,2,4.4,1.3,-1.5,3.1,56.8,928.8,0.0,8,0.0,0.0,0,0
2,349190625,ZAKOPANE,2020,1,3,2.5,1.4,-1.1,3.8,68.8,922.4,0.0,8,0.0,0.0,0,0
3,349190625,ZAKOPANE,2020,1,4,7.6,3.6,0.8,5.4,83.6,917.5,0.0,8,1.9,6.5,1,1
4,349190625,ZAKOPANE,2020,1,5,7.9,2.8,-4.1,3.8,83.6,924.4,0.0,8,3.4,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,349190625,ZAKOPANE,2019,12,27,7.9,2.1,-2.0,4.8,91.3,920.5,0.0,8,4.6,6.5,1,1
543,349190625,ZAKOPANE,2019,12,28,8.0,2.6,-4.6,4.0,90.6,928.0,0.0,8,3.7,0.4,1,1
544,349190625,ZAKOPANE,2019,12,29,7.1,0.9,-7.3,3.2,88.3,933.2,0.0,8,1.1,0.0,1,0
545,349190625,ZAKOPANE,2019,12,30,6.0,1.3,-2.3,4.8,87.5,931.0,0.0,8,0.0,0.0,0,0


Usuwamy kolumny 'Status pomiaru PPPM' oraz 'Średnie dobowe ciśnienie na poziomie morza'. Obie te kolumny zawierają wartości, które mają takie same wartości (nic nie wnoszą).

In [5]:
df = df.drop(df.columns[11],axis = 1)
df = df.drop(df.columns[11],axis = 1)

df

,Kod stacji,Nazwa stacji,Rok,Miesiąc,Dzień,Średnie dobowe zachmurzenie ogólne [oktanty],Średnia dobowa prędkość wiatru [m/s],Średnia dobowa temperatura [°C],Średnia dobowe ciśnienie pary wodnej [hPa],Średnia dobowa wilgotność względna [%],Średnia dobowe ciśnienie na poziomie stacji [hPa],Suma opadu dzień [mm],Suma opadu noc [mm],deszcz w dzień,deszcz w nocy
0,349190625,ZAKOPANE,2020,1,1,4.3,2.0,-1.7,4.0,73.1,929.5,0.0,0.0,0,0
1,349190625,ZAKOPANE,2020,1,2,4.4,1.3,-1.5,3.1,56.8,928.8,0.0,0.0,0,0
2,349190625,ZAKOPANE,2020,1,3,2.5,1.4,-1.1,3.8,68.8,922.4,0.0,0.0,0,0
3,349190625,ZAKOPANE,2020,1,4,7.6,3.6,0.8,5.4,83.6,917.5,1.9,6.5,1,1
4,349190625,ZAKOPANE,2020,1,5,7.9,2.8,-4.1,3.8,83.6,924.4,3.4,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,349190625,ZAKOPANE,2019,12,27,7.9,2.1,-2.0,4.8,91.3,920.5,4.6,6.5,1,1
543,349190625,ZAKOPANE,2019,12,28,8.0,2.6,-4.6,4.0,90.6,928.0,3.7,0.4,1,1
544,349190625,ZAKOPANE,2019,12,29,7.1,0.9,-7.3,3.2,88.3,933.2,1.1,0.0,1,0
545,349190625,ZAKOPANE,2019,12,30,6.0,1.3,-2.3,4.8,87.5,931.0,0.0,0.0,0,0


Przeprowadzamy konwersję kolumn Rok, Miesiąc, Dzień do formatu daty.

In [6]:
date = pd.to_datetime((df['Rok']*10000+df['Miesiąc']*100+df['Dzień']).apply(str),format='%Y%m%d')
df.insert(loc=2, column='Data', value=date)
for i in range(0,3):
    df = df.drop(df.columns[3],axis = 1)
df

,Kod stacji,Nazwa stacji,Data,Średnie dobowe zachmurzenie ogólne [oktanty],Średnia dobowa prędkość wiatru [m/s],Średnia dobowa temperatura [°C],Średnia dobowe ciśnienie pary wodnej [hPa],Średnia dobowa wilgotność względna [%],Średnia dobowe ciśnienie na poziomie stacji [hPa],Suma opadu dzień [mm],Suma opadu noc [mm],deszcz w dzień,deszcz w nocy
0,349190625,ZAKOPANE,2020-01-01,4.3,2.0,-1.7,4.0,73.1,929.5,0.0,0.0,0,0
1,349190625,ZAKOPANE,2020-01-02,4.4,1.3,-1.5,3.1,56.8,928.8,0.0,0.0,0,0
2,349190625,ZAKOPANE,2020-01-03,2.5,1.4,-1.1,3.8,68.8,922.4,0.0,0.0,0,0
3,349190625,ZAKOPANE,2020-01-04,7.6,3.6,0.8,5.4,83.6,917.5,1.9,6.5,1,1
4,349190625,ZAKOPANE,2020-01-05,7.9,2.8,-4.1,3.8,83.6,924.4,3.4,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,349190625,ZAKOPANE,2019-12-27,7.9,2.1,-2.0,4.8,91.3,920.5,4.6,6.5,1,1
543,349190625,ZAKOPANE,2019-12-28,8.0,2.6,-4.6,4.0,90.6,928.0,3.7,0.4,1,1
544,349190625,ZAKOPANE,2019-12-29,7.1,0.9,-7.3,3.2,88.3,933.2,1.1,0.0,1,0
545,349190625,ZAKOPANE,2019-12-30,6.0,1.3,-2.3,4.8,87.5,931.0,0.0,0.0,0,0


In [7]:
df.sort_values(by=['Data'], inplace=True)
df

,Kod stacji,Nazwa stacji,Data,Średnie dobowe zachmurzenie ogólne [oktanty],Średnia dobowa prędkość wiatru [m/s],Średnia dobowa temperatura [°C],Średnia dobowe ciśnienie pary wodnej [hPa],Średnia dobowa wilgotność względna [%],Średnia dobowe ciśnienie na poziomie stacji [hPa],Suma opadu dzień [mm],Suma opadu noc [mm],deszcz w dzień,deszcz w nocy
182,349190625,ZAKOPANE,2019-01-01,6.6,2.8,-1.0,5.0,84.9,918.4,0.3,3.0,1,1
183,349190625,ZAKOPANE,2019-01-02,7.8,3.3,-3.7,4.2,89.8,911.3,8.2,4.5,1,1
184,349190625,ZAKOPANE,2019-01-03,8.0,1.6,-7.5,3.0,85.5,919.1,6.5,2.7,1,1
185,349190625,ZAKOPANE,2019-01-04,7.9,1.5,-8.0,2.7,81.3,922.1,0.1,2.9,1,1
186,349190625,ZAKOPANE,2019-01-05,7.9,1.6,-3.4,4.3,90.5,912.7,8.0,1.3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,349190625,ZAKOPANE,2020-06-26,6.4,1.1,17.7,14.4,74.0,921.9,1.1,1.2,1,1
178,349190625,ZAKOPANE,2020-06-27,5.9,2.3,21.2,14.1,56.9,918.7,0.0,1.3,0,1
179,349190625,ZAKOPANE,2020-06-28,3.6,1.5,19.6,15.8,71.9,919.0,0.0,0.0,0,0
180,349190625,ZAKOPANE,2020-06-29,4.9,1.5,17.7,14.8,76.5,915.3,3.6,0.3,1,1


Za pomocą szeregów czasowych przewidujemy wartości każdego z parametrów na następne 7 dni. Następnie scalamy to w DataFrame.

In [8]:
#przewidywanie przyszłych parametrów
def make_predict(nazwa_kolumny,ilosc_dni):
    df2 = df[['Data',nazwa_kolumny]]
    df2.index = pd.DatetimeIndex(df2['Data'])
    model = ARMA(df2[nazwa_kolumny], order=(4, 2))
    model_fit = model.fit(disp=False)
    forecast = model_fit.predict(len(df2),len(df2)+ilosc_dni)
    return forecast[:][:]
#stworzenie zbioru danych z przewidywanymi parametrami

predicted_data = [] 
predicted_data.append(make_predict('Średnie dobowe zachmurzenie ogólne [oktanty]',7))
predicted_data.append(make_predict('Średnia dobowa prędkość wiatru [m/s]',7))
predicted_data.append(make_predict('Średnia dobowa temperatura [°C]',7))
predicted_data.append(make_predict('Średnia dobowe ciśnienie pary wodnej [hPa]',7))
predicted_data.append(make_predict('Średnia dobowa wilgotność względna [%]',7))
predicted_data.append(make_predict('Średnia dobowe ciśnienie na poziomie stacji [hPa]',7))

/home/albert/.local/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/home/albert/.local/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/home/albert/.local/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/home/albert/.local/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/home/albert/.local/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, Valu

In [9]:
df1 = pd.DataFrame(predicted_data, index=['Średnie dobowe zachmurzenie ogólne [oktanty]','Średnia dobowa prędkość wiatru [m/s]','Średnia dobowa temperatura [°C]','Średnia dobowe ciśnienie pary wodnej [hPa]','Średnia dobowa wilgotność względna [%]','Średnia dobowe ciśnienie na poziomie stacji [hPa]'])
df1 = df1.T
df1

,Średnie dobowe zachmurzenie ogólne [oktanty],Średnia dobowa prędkość wiatru [m/s],Średnia dobowa temperatura [°C],Średnia dobowe ciśnienie pary wodnej [hPa],Średnia dobowa wilgotność względna [%],Średnia dobowe ciśnienie na poziomie stacji [hPa]
2020-07-01,4.296043,1.733466,16.381971,11.765728,68.369018,917.199449
2020-07-02,5.094571,1.624096,16.098008,12.436361,71.960170,917.384980
2020-07-03,5.197881,1.607147,15.965382,12.838714,74.172265,916.979461
2020-07-04,5.407627,1.594438,15.908032,12.796538,74.859559,917.309048
2020-07-05,5.347958,1.606690,15.862905,12.728555,75.579014,917.441080
2020-07-06,5.439300,1.600912,15.817733,12.596476,75.530651,917.219927
2020-07-07,5.378912,1.612861,15.768035,12.552744,75.883523,917.309627
2020-07-08,5.432831,1.607452,15.716218,12.474294,75.658874,917.407322


Normalizujemy DataFrame'y metodą min-max.

In [10]:
columns_to_normalize = [[5],[6],[8],[9],[10]]
normalize_particular_columns(df, columns_to_normalize)
columns_to_normalize = [[5],[0],[1],[2],[3],[4]]
normalize_particular_columns(df1, columns_to_normalize)

Dzielimy dane na zbiory treningowe i testowe.

In [11]:
X = df[[df.columns[3],df.columns[4],df.columns[5],df.columns[6],df.columns[7],df.columns[8]]]
y = df[[df.columns[11]]]

In [12]:
Xtrain, Xtest, ytrain, ytest  = train_test_split(X,y,test_size=0.2,random_state=75)

## Model SVN 

In [13]:
clf= svm.SVC(kernel='linear', C=1.0)
clf.fit(Xtrain,ytrain)

SVC(kernel='linear')

In [36]:
ypred = clf.predict(Xtest)
clf_score = clf.score(Xtest,ytest)
print(f"Dokładoność modelu SVN wynosi {clf_score:.2f}")

Dokładoność modelu SVN wynosi 0.79


In [16]:
confusion_matrix(ytest,ypred)

array([[54,  5],
       [18, 33]])

## Model logistic regresion

In [37]:
logistic_regression= LogisticRegression()
logistic_regression.fit(Xtrain,ytrain)
y_pred_r=logistic_regression.predict(Xtest)
log_accur = accuracy_score(ytest, y_pred_r)
print(f"Dokładoność modelu regresji logistycznej wynosi {log_accur:.2f}")

Dokładoność modelu regresji logistycznej wynosi 0.80


In [33]:
confusion_matrix(ytest,y_pred_r)

array([[55,  4],
       [18, 33]])

## Model KNN

In [38]:
knn = KNeighborsClassifier()
knn.fit(Xtrain, ytrain)
y_pred_knn = knn.predict(Xtest)
accurKNN = accuracy_score(ytest, y_pred_knn)

print(f"Dokładoność modelu kNN wynosi {accurKNN:.2f}")

Dokładoność modelu kNN wynosi 0.73


In [35]:
confusion_matrix(ytest,y_pred_knn)

array([[47, 12],
       [18, 33]])

### Test danych przewidzianych

In [25]:
result_knn = knn.predict(df1)
result_log = logistic_regression.predict(df1)
result_svn = clf.predict(df1)

In [26]:
result_knn

array([0, 0, 0, 0, 0, 0, 0, 0])

In [27]:
result_log

array([0, 0, 0, 0, 0, 0, 0, 0])

In [28]:
result_svn

array([0, 0, 0, 0, 0, 0, 0, 0])

In [29]:
target_names = ['brak deszczu', 'deszcz']
report = classification_report(ytest, y_pred_knn, target_names= target_names)
print(report)

precision    recall  f1-score   support

brak deszczu       0.72      0.80      0.76        59
      deszcz       0.73      0.65      0.69        51

    accuracy                           0.73       110
   macro avg       0.73      0.72      0.72       110
weighted avg       0.73      0.73      0.73       110

